<a href="https://colab.research.google.com/github/PhuongUyen-tt/blank-app/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# prompt: kết nối với gg drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# prompt: đọc file theo đường dẫn: /content/drive/MyDrive/HK II | 24.25/AI in Biz /BTL /data.csv

import pandas as pd

file_path = '/content/drive/MyDrive/HK II | 24.25/AI in Biz /BTL /data.csv'
df = pd.read_csv(file_path)
print(df.head())

   transaction_id     transaction_date transaction_time  store_id  \
0            1567  2023-03-19 00:00:00         20:30:42       501   
1            1568  2022-07-14 00:00:00         22:59:16       302   
2            1569  2023-05-03 00:00:00         15:31:33       201   
3            1570  2025-04-11 00:00:00         15:30:54       202   
4            1571  2024-02-20 00:00:00         09:25:54       401   

            store_location  product_id  transaction_qty  unit_price  \
0  Nha Trang - City Center           7                2       45000   
1      Da Nang - Thanh Khe           1                1       29000   
2        Hanoi - Hoan Kiem          24                4       35000   
3          Hanoi - Ba Dinh          15                5       55000   
4        Hue - City Center           3                2       29000   

   Total_Bill    product_category product_type    product_detail     Size  \
0       90000                 Tea          Tea      Tra Sen Vang        M   
1   

In [51]:
# prompt: chia tập dữ liệu thành 2 tập train và tập test với test sz 0.2

from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
# X will be the features (all columns except 'target_column')
# y will be the target variable ('target_column')
# Replace 'target_column' with the actual name of your target column
# If your data doesn't have a specific target column and you are splitting for unsupervised learning,
# you might just split the entire dataframe X = df
if 'target_column' in df.columns:
  X = df.drop('target_column', axis=1)
  y = df['target_column']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  print("\nData split into training and testing sets:")
  print("X_train shape:", X_train.shape)
  print("X_test shape:", X_test.shape)
  print("y_train shape:", y_train.shape)
  print("y_test shape:", y_test.shape)
else:
  print("\nAssuming unsupervised splitting (no target column specified).")
  X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)
  print("\nData split into training and testing sets:")
  print("X_train shape:", X_train.shape)
  print("X_test shape:", X_test.shape)




Assuming unsupervised splitting (no target column specified).

Data split into training and testing sets:
X_train shape: (24000, 20)
X_test shape: (6000, 20)


In [52]:
# prompt: Preprocess data, train a model to predict "product_category" from key features " store_location".,"Gender", "Age'", "Total_Bill", "Season", "Size", "transaction_qty" with Gradient Boosting Classification model

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Define features and target
features = ["store_location", "Gender", "Age", "Total_Bill", "Season", "Size", "transaction_qty"]
target = "product_category"

# Check if the target column exists
if target not in df.columns:
    print(f"Error: Target column '{target}' not found in the DataFrame.")
else:
    # Check if all features exist
    missing_features = [feat for feat in features if feat not in df.columns]
    if missing_features:
        print(f"Error: Missing feature columns: {missing_features}")
    else:
        # Preprocessing: Handle categorical features
        X = df[features].copy()
        y = df[target].copy()

        # Use Label Encoding for simplicity on categorical features
        categorical_features = ["store_location", "Gender", "Season", "Size"]
        for col in categorical_features:
            if col in X.columns:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col])

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train Gradient Boosting Classifier
        gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        gb_model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = gb_model.predict(X_test)

        # Evaluate the model
        print("\nGradient Boosting Classifier Results:")
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        # Optional: Print feature importances
        print("\nFeature Importances:")
        feature_importances = pd.Series(gb_model.feature_importances_, index=features).sort_values(ascending=False)
feature_importances


Gradient Boosting Classifier Results:
Classification Report:
                       precision    recall  f1-score   support

Espresso Based Coffee       0.57      0.48      0.52      1381
                 Food       1.00      1.00      1.00      1396
               Freeze       0.70      0.58      0.64      1199
                  Tea       0.66      1.00      0.79       813
   Traditional Coffee       1.00      1.00      1.00      1211

             accuracy                           0.80      6000
            macro avg       0.79      0.81      0.79      6000
         weighted avg       0.80      0.80      0.79      6000


Confusion Matrix:
[[ 667    0  290  424    0]
 [   0 1396    0    0    0]
 [ 506    0  693    0    0]
 [   0    0    0  813    0]
 [   0    0    0    0 1211]]

Feature Importances:


,0
Size,0.487382
Total_Bill,0.374039
transaction_qty,0.136607
Age,0.000798
store_location,0.000587
Season,0.000383
Gender,0.000206


In [53]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder

# Select a subset of potentially relevant columns for feature engineering
selected_cols = ["store_location", "Gender", "Age", "Total_Bill", "Season", "Size", "transaction_qty"]
# Assuming df is already loaded from previous cells
# df = pd.read_csv(file_path) # If df is not available, uncomment and load your data here
df_subset = df[selected_cols].copy()

# Identify categorical columns in the subset and apply one-hot encoding
categorical_cols_subset = df_subset.select_dtypes(include=['object']).columns
df_subset = pd.get_dummies(df_subset, columns=categorical_cols_subset, drop_first=True)

# Define features (X) using the subset DataFrame
X = df_subset

# Define and encode the target variable (y)
y = df['product_category'] # Use the original 'product_category' as the target

# Use LabelEncoder to convert the target variable (product_category) into numerical labels
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

# Split the data into training and testing sets using the encoded target
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Instantiate the XGBoost Regressor model
# Since you are predicting a categorical target, you should use XGBClassifier instead of XGBRegressor
xgb_model = xgb.XGBClassifier(random_state=42) # Changed to XGBClassifier

# Define a smaller parameter distribution for RandomizedSearchCV
# Adjust parameters for a classifier if needed, but the current range might be okay
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.05),
    'max_depth': randint(3, 7),
    'min_child_weight': randint(1, 4),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2)
}

# Instantiate RandomizedSearchCV with reduced cv and n_jobs=1
# Change scoring to 'accuracy' or other classification metrics
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist,
                                   n_iter=30, scoring='accuracy', cv=2, n_jobs=1, verbose=2, random_state=42) # Changed scoring to 'accuracy'

# Fit RandomizedSearchCV to the training data using the encoded target
print("Performing Randomized Search for Hyperparameter Tuning...")
random_search.fit(X_train, y_train_encoded) # Use y_train_encoded

# Print the best hyperparameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("\nBest hyperparameters found: ", best_params)
print("\nBest Cross-Validation Accuracy:") # Changed label for scoring
print(best_score)

# You might want to evaluate the best model on the test set using appropriate classification metrics
# For example:
# best_xgb_model = random_search.best_estimator_
# y_pred_encoded = best_xgb_model.predict(X_test)
# from sklearn.metrics import accuracy_score
# test_accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
# print("\nTest Accuracy with Best Parameters:", test_accuracy)

Performing Randomized Search for Hyperparameter Tuning...
Fitting 2 folds for each of 30 candidates, totalling 60 fits
[CV] END colsample_bytree=0.8749080237694725, learning_rate=0.05753571532049581, max_depth=5, min_child_weight=1, n_estimators=70, subsample=0.8312037280884873; total time=   0.6s
[CV] END colsample_bytree=0.8749080237694725, learning_rate=0.05753571532049581, max_depth=5, min_child_weight=1, n_estimators=70, subsample=0.8312037280884873; total time=   3.2s
[CV] END colsample_bytree=0.8311989040672406, learning_rate=0.012904180608409973, max_depth=6, min_child_weight=1, n_estimators=149, subsample=0.8285733635843882; total time=   5.9s
[CV] END colsample_bytree=0.8311989040672406, learning_rate=0.012904180608409973, max_depth=6, min_child_weight=1, n_estimators=149, subsample=0.8285733635843882; total time=   1.3s
[CV] END colsample_bytree=0.9301776945897706, learning_rate=0.012820578951355013, max_depth=6, min_child_weight=2, n_estimators=87, subsample=0.8001557531682

## Streamlit app development

### Subtask:
Build a Streamlit application that takes seasonal data as input and uses the trained model to predict product demand.


**Reasoning**:
Create the `app.py` file with the necessary imports, load the best model, and set up the basic Streamlit app structure with input widgets.



In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import os # Import os for checking file existence

# Load the trained XGBoost model
model_path = 'xgboost_model.pkl'
if not os.path.exists(model_path):
    st.error(f"Model file '{model_path}' not found. Please train and save the model first.")
    st.stop()
try:
    model = joblib.load(model_path)
except Exception as e:
    st.error(f"Error loading the model file: {e}")
    st.stop()

# Load the feature columns used during training
features_path = 'xgboost_features.pkl'
if not os.path.exists(features_path):
    st.error(f"Feature columns file '{features_path}' not found. Please run the training steps first to save X_train columns.")
    st.stop()
try:
    expected_cols = joblib.load(features_path)
except Exception as e:
    st.error(f"Error loading the feature columns file: {e}")
    st.stop()

# Load the original dataframe to get unique values for dropdowns
# This is needed to populate selectboxes with actual data values
file_path = '/content/drive/MyDrive/Phú Quốc/du_lieu_da_them_season_va_income( dữ liệu cuối cùng).xlsx - Sheet1.csv'
try:
    original_df = pd.read_csv(file_path)
    unique_store_ids = sorted(original_df['store_id'].unique().tolist())
    unique_store_locations = sorted(original_df['store_location'].unique().tolist())
    unique_product_ids = sorted(original_df['product_id'].unique().tolist())
    unique_occupations = sorted(original_df['Occupation'].unique().tolist())
    unique_seasons = sorted(original_df['Season'].unique().tolist())
    unique_genders = sorted(original_df['Gender'].unique().tolist())

except FileNotFoundError:
    st.error(f"Original data file not found at {file_path}. Please ensure the file exists.")
    st.stop()
except KeyError as e:
    st.error(f"Missing expected column in original data file: {e}")
    st.stop()
except Exception as e:
    st.error(f"Error loading original data: {e}")
    st.stop()


st.title("Seasonal Product Demand Forecasting")
st.header("Input the details to predict product demand for a season.")

# Define input widgets based on the features used for training (X_train)
# Features used in X_train were derived from:
# 'store_id', 'product_id', 'transaction_qty', 'unit_price', 'Age', 'Income',
# 'Season', 'Occupation', 'month', 'year'
# plus one-hot encoded Gender and Store Location (if added during training)

store_id = st.selectbox("Store ID", unique_store_ids)
store_location = st.selectbox("Store Location", unique_store_locations)
product_id = st.selectbox("Product ID", unique_product_ids)

transaction_qty = st.number_input("Transaction Quantity", min_value=1, value=1, step=1)
unit_price = st.number_input("Unit Price", min_value=0, value=int(original_df['unit_price'].median()), step=1000) # Use median from data, cast to int
age = st.number_input("Age", min_value=0, max_value=120, value=int(original_df['Age'].mean()), step=1) # Use mean from data
income = st.number_input("Income", min_value=0, value=int(original_df['Income'].median()), step=100000) # Use median from data

gender = st.selectbox("Gender", unique_genders)
season = st.selectbox("Season", unique_seasons)
occupation = st.selectbox("Occupation", unique_occupations)

month = st.number_input("Month (1-12)", min_value=1, max_value=12, value=6, step=1)
year = st.number_input("Year", min_value=2022, max_value=2030, value=2024, step=1)


# Button to trigger prediction
predict_button = st.button("Predict Demand")

if predict_button:
    # Create a DataFrame from user inputs
    input_data = {
        'store_id': store_id,
        'store_location': store_location,
        'product_id': product_id,
        'transaction_qty': transaction_qty,
        'unit_price': unit_price,
        'Age': age,
        'Income': income,
        'Gender': gender,
        'month': month,
        'year': year,
        'Season': season,
        'Occupation': occupation
    }
    input_df = pd.DataFrame([input_data])

    # Apply one-hot encoding to categorical columns in input_df to match expected_cols
    # We need to apply one-hot encoding to 'Season', 'Occupation', 'Gender', and 'store_location'

    categorical_cols_to_encode = ['Season', 'Occupation', 'Gender', 'store_location'] # Add 'store_location'

    # Apply one-hot encoding to the input DataFrame
    # Use the unique values from the original data to ensure all possible categories are covered
    input_df = pd.get_dummies(input_df, columns=['Season'], prefix='Season', drop_first=True)
    input_df = pd.get_dummies(input_df, columns=['Occupation'], prefix='Occupation', drop_first=True)
    input_df = pd.get_dummies(input_df, columns=['Gender'], prefix='Gender', drop_first=True)
    input_df = pd.get_dummies(input_df, columns=['store_location'], prefix='store_location', drop_first=True)


    # Reindex the input_df to match the expected columns from training and fill missing columns with 0
    # This step is crucial to ensure the input data has the same columns in the same order as X_train
    # expected_cols is loaded from 'xgboost_features.pkl'

    input_df = input_df.reindex(columns=expected_cols, fill_value=0)

    # Make prediction
    try:
        predicted_demand = model.predict(input_df)
        st.success(f"Predicted Product Demand: {predicted_demand[0]:,.2f} VND")
    except Exception as e:
        st.error(f"An error occurred during prediction: {e}")

st.write("To run this app, save the code as `app.py` and run `streamlit run app.py` in your terminal or use the ngrok tunnel.")

Overwriting app.py


In [54]:
# prompt: Build a Streamlit application that takes "store_location", "Gender", "Age", "Total_Bill", "Season", "Size", "transaction_qty" as input and uses the trained model to predict  suggested product category

# Save the trained Gradient Boosting model
model_path_gb = 'gradient_boosting_model.pkl'
joblib.dump(gb_model, model_path_gb)
print(f"\nGradient Boosting model saved to {model_path_gb}")

# Save the LabelEncoder used for the target variable (product_category)
label_encoder_target_path = 'product_category_label_encoder.pkl'
joblib.dump(label_encoder_y, label_encoder_target_path) # Use the label_encoder_y defined earlier for 'product_category'
print(f"Target LabelEncoder saved to {label_encoder_target_path}")

# Save the list of feature columns used for training the GB model
# The GB model used the original 'features' list after applying LabelEncoding
features_path_gb = 'gradient_boosting_features.pkl'
joblib.dump(features, features_path_gb)
print(f"Feature columns saved to {features_path_gb}")



Gradient Boosting model saved to gradient_boosting_model.pkl
Target LabelEncoder saved to product_category_label_encoder.pkl
Feature columns saved to gradient_boosting_features.pkl


In [62]:
# prompt: save the code as `app.py`

%%writefile app.py
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder
import streamlit as st
import joblib
import os # Import os for checking file existence

# Mount Google Drive (necessary for Colab execution, remove for local Streamlit)
# drive.mount('/content/drive')

# --- File Paths ---
# Update these paths to reflect the correct location of your data and saved files
file_path_data = '/content/drive/MyDrive/HK II | 24.25/AI in Biz /BTL /data.csv' # Path to your training data
# file_path_data = '/content/drive/MyDrive/Phú Quốc/du_lieu_da_them_season_va_income( dữ liệu cuối cùng).xlsx - Sheet1.csv' # Path to your data for unique values
model_path = 'xgboost_model.pkl'
features_path = 'xgboost_features.pkl'
label_encoder_target_path = 'product_category_label_encoder.pkl'

# --- Load Data and Setup ---
# Load the original dataframe to get unique values for dropdowns
# This is needed to populate selectboxes with actual data values
try:
    original_df = pd.read_csv(file_path_data)
    # Extract unique values for dropdowns. Adjust column names as per your data.
    # Ensure these column names match the features used in the model.
    unique_store_locations = sorted(original_df['store_location'].dropna().unique().tolist())
    unique_seasons = sorted(original_df['Season'].dropna().unique().tolist())
    unique_genders = sorted(original_df['Gender'].dropna().unique().tolist())
    # Add other unique columns relevant to your features here
    # Example: unique_occupations = sorted(original_df['Occupation'].dropna().unique().tolist())
    # Example: unique_product_categories = sorted(original_df['product_category'].dropna().unique().tolist()) # For target decoding later if needed

except FileNotFoundError:
    st.error(f"Original data file not found at {file_path_data}. Please ensure the file exists.")
    st.stop()
except KeyError as e:
    st.error(f"Missing expected column in original data file: {e}")
    st.stop()
except Exception as e:
    st.error(f"Error loading original data: {e}")
    st.stop()

# --- Load Trained Model and Artifacts ---
# Load the trained XGBoost model
if not os.path.exists(model_path):
    st.error(f"Model file '{model_path}' not found. Please train and save the model first.")
    st.stop()
try:
    model = joblib.load(model_path)
except Exception as e:
    st.error(f"Error loading the model file: {e}")
    st.stop()

# Load the feature columns used during training
if not os.path.exists(features_path):
    st.error(f"Feature columns file '{features_path}' not found. Please run the training steps first to save X_train columns.")
    st.stop()
try:
    expected_cols = joblib.load(features_path)
except Exception as e:
    st.error(f"Error loading the feature columns file: {e}")
    st.stop()

# Load the LabelEncoder for the target variable if you need to decode predictions
# if not os.path.exists(label_encoder_target_path):
#     st.warning(f"Target LabelEncoder file '{label_encoder_target_path}' not found. Predictions will be numerical.")
#     label_encoder_y = None
# else:
#     try:
#         label_encoder_y = joblib.load(label_encoder_target_path)
#     except Exception as e:
#         st.error(f"Error loading the Target LabelEncoder file: {e}")
#         label_encoder_y = None


# --- Streamlit App Layout ---
st.title("Seasonal Product Demand Forecasting")
st.header("Input the details to predict product demand for a season.")

# --- Input Widgets ---
# Define input widgets based on the features used for training (expected_cols)
# You need to map the feature columns in expected_cols back to meaningful user inputs.
# For one-hot encoded features like Season_Spring, you'll need a single 'Season' selectbox.

# Example based on the features you mentioned: ["store_location", "Gender", "Age", "Total_Bill", "Season", "Size", "transaction_qty"]
# And the ones used for XGBoost: derived from "store_location", "Gender", "Age", "Total_Bill", "Season", "Size", "transaction_qty" after one-hot encoding

st.subheader("Enter Product and Customer Details:")

# Use dropdowns for categorical features
store_location = st.selectbox("Store Location", unique_store_locations)
gender = st.selectbox("Gender", unique_genders)
season = st.selectbox("Season", unique_seasons)
# Assuming 'Size' is also a categorical feature based on your previous code snippet
# You would need unique values for 'Size' from your original data
# unique_sizes = sorted(original_df['Size'].dropna().unique().tolist())
# size = st.selectbox("Size", unique_sizes)


# Use number inputs for numerical features
age = st.number_input("Age", min_value=0, max_value=120, value=30, step=1) # Default to a reasonable value
total_bill = st.number_input("Total Bill", min_value=0.0, value=100.0, step=0.1) # Default to a reasonable value
transaction_qty = st.number_input("Transaction Quantity", min_value=1, value=1, step=1)


# Button to trigger prediction
predict_button = st.button("Predict Demand")

# --- Prediction Logic ---
if predict_button:
    # Create a DataFrame from user inputs
    input_data = {
        'Age': age,
        'Total_Bill': total_bill,
        'transaction_qty': transaction_qty,
        'Gender': gender,
        'Season': season,
        # 'Size': size, # Include if 'Size' is a feature
        'store_location': store_location,
        # Add other numerical features here if used in your model training
        # e.g., 'Income', 'unit_price', 'month', 'year'
    }
    input_df = pd.DataFrame([input_data])

    # Apply one-hot encoding to the categorical columns in the input DataFrame
    # This must match the encoding applied to the training data (X_train)
    categorical_cols_to_encode_input = ['Gender', 'Season', 'store_location'] # Add 'Size' if used
    # Ensure the column names match your input_df keys

    # Apply one-hot encoding. This requires knowing the unique categories seen during training.
    # A safer way is to use the original_df to get all possible categories.
    # This ensures that even if a category is not in the current input, the column exists with value 0.

    input_df = pd.get_dummies(input_df, columns=['Gender'], prefix='Gender', drop_first=False) # Keep all dummies for reindexing later
    input_df = pd.get_dummies(input_df, columns=['Season'], prefix='Season', drop_first=False) # Keep all dummies
    input_df = pd.get_dummies(input_df, columns=['store_location'], prefix='store_location', drop_first=False) # Keep all dummies
    # input_df = pd.get_dummies(input_df, columns=['Size'], prefix='Size', drop_first=False) # Include if Size is used

    # Reindex the input_df to match the expected columns from training (expected_cols)
    # Fill missing columns (due to one-hot encoding not generating all possible columns in the input) with 0
    input_df_processed = input_df.reindex(columns=expected_cols, fill_value=0)

    # Ensure the order of columns matches the training data
    input_df_processed = input_df_processed[expected_cols]


    # Make prediction
    try:
        # Predict the numerical product category label
        predicted_label = model.predict(input_df_processed)[0]

        # Decode the predicted label back to the original product category string
        # This requires the LabelEncoder used during training
        # If label_encoder_y is None, we can only display the numerical label
        # if label_encoder_y:
        #     predicted_category = label_encoder_y.inverse_transform([int(predicted_label)])[0]
        #     st.success(f"Predicted Product Category: **{predicted_category}**")
        # else:
        st.success(f"Predicted Product Category (Numerical Label): **{int(predicted_label)}**") # Display as integer


    except Exception as e:
        st.error(f"An error occurred during prediction: {e}")
        st.write("Please check the input values and ensure the model and feature files are correctly loaded.")

st.write("To run this app:")
st.code("1. Save the code above as `app.py`")
st.code("2. Ensure 'xgboost_model.pkl' and 'xgboost_features.pkl' are in the same directory or accessible path.")
st.code("3. Open your terminal or command prompt.")
st.code("4. Navigate to the directory where you saved `app.py`.")
st.code("5. Run the command: `streamlit run app.py`")
st.write("If running on Google Colab, you'll need to use ngrok or another tunneling service to expose the Streamlit app.")

# Note: The training steps (Gradient Boosting and XGBoost with RandomizedSearchCV)
# are still included in this file. When running `streamlit run app.py` locally,
# these training steps will execute every time the app starts, which is inefficient.
# It's recommended to perform training and model saving in a separate script or notebook,
# and only load the saved model and artifacts in the Streamlit app.

# Example of saving the trained XGBoost model and feature columns after training (in your training script/notebook):
# Assuming `random_search` has been fitted and `expected_cols` are the columns of X_train

# # Save the best XGBoost model
# best_xgb_model = random_search.best_estimator_
# joblib.dump(best_xgb_model, model_path)
# print(f"\nBest XGBoost model saved to {model_path}")

# # Save the list of feature columns
# joblib.dump(X_train.columns.tolist(), features_path)
# print(f"Feature columns list saved to {features_path}")

# # Save the target LabelEncoder
# joblib.dump(label_encoder_y, label_encoder_target_path)
# print(f"Target LabelEncoder saved to {label_encoder_target_path}")

```

Overwriting app.py
